In [1]:
import pandas as pd
import numpy as np

file_path = 'data.csv'
df = pd.read_csv(file_path,encoding='utf-8')

In [2]:
df.head()

,language,proj_id,file_id,file_body
0,JavaScript,10001,100001,// Functions as values of a variable\nvar cube...
1,JavaScript,10001,100002,// Functions as values of a variable\nvar cube...
2,JavaScript,10002,100003,function median(ary) {\n if (ary.length == ...
3,JavaScript,10002,100004,"[\n null,\n 4,\n 3.5,\n 2.1\n]\n"
4,JavaScript,10002,100005,(() => {\n 'use strict';\n\n // median :...


In [3]:
languages=set(df['language'])
keys=dict()
i=0
for lang in languages:
    keys.update({lang:i})
    i+=1

In [4]:
keys

{'Fortran': 0,
 'Julia': 1,
 'Perl': 2,
 'Rust': 3,
 'MATLAB': 4,
 'Haskell': 5,
 'C': 6,
 'Kotlin': 7,
 'Ruby': 8,
 'Scala': 9,
 'Python': 10,
 'PHP': 11,
 'Mathematica': 12,
 'JavaScript': 13,
 'C++': 14,
 'Swift': 15,
 'R': 16,
 'Java': 17,
 'Go': 18}

In [9]:
out = df['language']
text = df['file_body']

label = []
text2=[]
for val,t in zip(out,text):
#     if val!='Python': #problems with numpy.nan
        label.append(keys[val])
        text2.append(t)

label = np.array(label)
text = np.array(text2)

label

array([13, 13, 13, ...,  2,  2,  2])

In [10]:
label.shape

(16272,)

In [11]:
text.shape

(16272,)

In [12]:
# from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

x_train = []
x_test = []
y_train = []
y_test = []

# text, label = shuffle(text,label,random_state=42)
x_train, x_test, y_train, y_test = train_test_split(text,label,train_size=0.9,shuffle=True,random_state=42)

x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

/home/filip/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [13]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

count_vect = CountVectorizer(decode_error='ignore')
x_train_count = count_vect.fit_transform(x_train)
tfidf_trans = TfidfTransformer()
x_train_tfidf = tfidf_trans.fit_transform(x_train_count)

x_test_count = count_vect.transform(x_test)
x_test_tfidf = tfidf_trans.transform(x_test_count)

In [14]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

clf = XGBClassifier(n_estimators=200)
clf.fit(x_train_tfidf,y_train)
y_pred = clf.predict(x_test_tfidf)
print(accuracy_score(y_test,y_pred))

0.8538083538083538


/home/filip/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
